# 주제 : 고객 세분화를 통한 혁신적 기업성장 솔루션 
## 데이터 분석 목표 설정 및 개요
1. 데이터 분석 목표
   +    1. 고객 세분화를 통해 차년도 매출향상 전략 탐색
   +    2. 성장 기반이 되는 잠재 요소 탐색
2. 개요
   + RFM 지표를 활용하여 K-Means Clustering을 진행하고 5가지 세그먼트로 고객을 분류한다.
   + 분류된 세그먼트를 다른 데이터와 비교하여 기업 성장에 필요한 요소를 탐색한다.
   + 분석을 바탕으로 기업성장 솔루션을 제시한다.

## \<목차\>
1. 세팅
   1. 필요 라이브러리 import
   2. 데이터 로드
2. EDA
   1. 데이터 기본 정보 확인(type, shape, null counts, uniques, etc..)
   2. 분석 데이터 통합을 위한 전처리
3. 데이터 분석(시각화)
   + 월별 매출/거래량 추이 분석
   + 월별 마케팅비용 추이 분석
   + 마케팅 비용과 매출과의 선형관계 분석
   + 가입기간과 매출 관계 분석
   + 가입연도별 신규고객동향 파악
   + 지역별 매출/거래량 분석
   + 제품카테고리별 매출/거래량 분석
4. 코호트 분석
   + 월별 코호트 분석
   + 잔존 빈도 분석
   + 잔존율 분석
   + 월별 매출액 리텐션 분석
5. RFM과 K-Means를 활용한 고객 세그먼트 정의    
   + RFM 계산
   + RFM 3D 그래프
   + 이상치를 제거한 고객의 RFM 3차원 그래프
   + RFM 값을 변수로 하는 K-Means 클러스터링 수행
   + QQ-Plot을 통한 정규성 검정
   + K-Means 하이퍼파라미터 K(엘보우, 실루엣) 서칭 & 클러스터링 
   + 클러스터링된 데이터의 RFM 3차원 그래프 분석  
   
6. 클러스터링된 데이터 분석
   + RFM 분석
   + 월 거래량 & 매출 분석
   + 고객군별 제품 카테고리 분석
   + 월별 제품 카테고리 분석
   + ARPU 분석
   + 고객군별 지역 분석
   + 고객군별 성별 분석
   + Q. 마케팅비용이 고객군별 매출/거래량에 어떤 영향을 끼칠까?
   + 쿠폰 분석
7. 결론

# 코드 시작

## 1. 기초 세팅
### 1-1. 필요 라이브러리 import

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', 100)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm # color map

plt.style.use('ggplot')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import datetime
from datetime import datetime
import calendar

from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox, kurtosis

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_score

import math


### 1-2. 데이터 로드
+ Onlinesales_info: 2019년도 한해 동안 발생한 온라인 거래와 관련된 정보보
+ Customer_info: 고객 관련 정보
+ Discount_info: 할인 관련 정보
+ Marketing_info: 마케팅 비용 관련 정보
+ Tax_info: 세금 관련 정보보

In [2]:
online_sales = pd.read_csv('data/Onlinesales_info.csv')
discount_coupon = pd.read_csv('data/Discount_info.csv')
customer_data = pd.read_csv('data/customer_info.csv')
marketing_spend = pd.read_csv('data/marketing_info.csv')
tax_amount = pd.read_csv('data/Tax_info.csv')

## 2. EDA
### 2-1. 데이터 기본 정보 확인

+ 타입 확인
+ 갯수 확인
+ 중복값 확인
+ 결측치 확인
+ 결측치 비율 확인
+ 고유값 확인


In [3]:
def str_summary(df, pred=None):
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()]).T.squeeze() # 각 feature의 고유값을 시리즈 타입으로 반환
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0]) # 각 feature의 고유값 개수
    missing_ratio = (df.isnull().sum() / obs) * 100
    
    print("Data shape: ", df.shape)

    cols = ['Types', 'Counts', 'Distincts', 'Nulls', 'Missing_ratio', 'Uniques']
    structure = pd.concat([types, counts, distincts, nulls, missing_ratio, uniques], axis=1, sort=True)

    structure.columns = cols

    print("================================================")
    print("Data types: ")
    print(structure['Types'].value_counts())
    print("================================================")
    print("\n\n")

    return structure

In [4]:
data_summary_1 = str_summary(online_sales)
data_summary_2 = str_summary(discount_coupon)
data_summary_3 = str_summary(customer_data)
data_summary_4 = str_summary(marketing_spend)
data_summary_5 = str_summary(tax_amount)

Data shape:  (52924, 9)
Data types: 
object     6
float64    2
int64      1
Name: Types, dtype: int64



Data shape:  (204, 4)
Data types: 
object    3
int64     1
Name: Types, dtype: int64



Data shape:  (1468, 4)
Data types: 
object    3
int64     1
Name: Types, dtype: int64



Data shape:  (365, 3)
Data types: 
object     1
int64      1
float64    1
Name: Types, dtype: int64



Data shape:  (20, 2)
Data types: 
float64    1
object     1
Name: Types, dtype: int64





In [5]:
display(data_summary_1)
display(data_summary_2)
display(data_summary_3)
display(data_summary_4)
display(data_summary_5)

,Types,Counts,Distincts,Nulls,Missing_ratio,Uniques
거래ID,object,52924,25061,0,0.0,"[Transaction_0000, Transaction_0001, Transaction_0002, Transaction_0003, Transaction_0004, Trans..."
거래날짜,object,52924,365,0,0.0,"[2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,..."
고객ID,object,52924,1468,0,0.0,"[USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07..."
배송료,float64,52924,267,0,0.0,"[6.5, 102.79, 28.78, 8.7, 20.0, 17.96, 24.47, 35.96, 18.47, 74.74, 35.3, 20.78, 13.78, 122.74, 1..."
수량,int64,52924,151,0,0.0,"[1, 5, 15, 52, 31, 2, 3, 26, 10, 4, 6, 57, 103, 62, 12, 41, 30, 21, 206, 516, 14, 155, 34, 258, ..."
제품ID,object,52924,1145,0,0.0,"[Product_0981, Product_0904, Product_0203, Product_0848, Product_0854, Product_0880, Product_088..."
제품카테고리,object,52924,20,0,0.0,"[Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu..."
쿠폰상태,object,52924,3,0,0.0,"[Used, Not Used, Clicked]"
평균금액,float64,52924,546,0,0.0,"[153.71, 2.05, 17.53, 16.5, 5.15, 3.08, 10.31, 9.27, 0.98, 1.99, 122.77, 81.5, 14.02, 10.72, 1.0..."


,Types,Counts,Distincts,Nulls,Missing_ratio,Uniques
월,object,204,12,0,0.0,"[Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec]"
제품카테고리,object,204,17,0,0.0,"[Apparel, Nest-USA, Office, Drinkware, Lifestyle, Bags, Notebooks, Headgear, Nest, Waze, Bottles..."
쿠폰코드,object,204,48,0,0.0,"[SALE10, SALE20, SALE30, ELEC10, ELEC20, ELEC30, OFF10, OFF20, OFF30, EXTRA10, EXTRA20, EXTRA30,..."
할인율,int64,204,3,0,0.0,"[10, 20, 30]"


,Types,Counts,Distincts,Nulls,Missing_ratio,Uniques
가입기간,int64,1468,49,0,0.0,"[12, 43, 33, 30, 49, 32, 46, 24, 40, 10, 19, 14, 25, 50, 39, 21, 29, 26, 41, 28, 15, 18, 31, 27,..."
고객ID,object,1468,1468,0,0.0,"[USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07..."
고객지역,object,1468,5,0,0.0,"[Chicago, California, New York, New Jersey, Washington DC]"
성별,object,1468,2,0,0.0,"[남, 여]"


,Types,Counts,Distincts,Nulls,Missing_ratio,Uniques
날짜,object,365,365,0,0.0,"[2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,..."
오프라인비용,int64,365,11,0,0.0,"[4500, 5000, 3000, 1000, 700, 3500, 4000, 2500, 2000, 500, 1500]"
온라인비용,float64,365,365,0,0.0,"[2424.5, 3480.36, 1576.38, 2928.55, 4055.3, 3796.85, 2579.52, 2551.38, 3287.83, 515.44, 1757.58,..."


,Types,Counts,Distincts,Nulls,Missing_ratio,Uniques
GST,float64,20,4,0,0.0,"[0.1, 0.18, 0.05, 0.12]"
제품카테고리,object,20,20,0,0.0,"[Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu..."


#### 5개의 데이터셋에는 결측치가 포함되어 있지 않다.
#### Onlinesale_info 특이사항
+ [고객ID]는 중복값이 포함되어 있다. 
    + 한명의 고객이 여러 구매건에 관여함. 
+ [거래ID]는 중복값이 포함되어 있고, 하나의 거래ID에 서로 다른 고객ID가 포함되어 있다. 
    + 거래 ID는 일정한 기간(몇 초) 안에 구매된 건별로 집계하는 것으로 보임.
+ [쿠폰상태]의 Clicked 데이터는 클릭은 했지만 결제에 사용하지 않은 데이터다.

### 2-2. 데이터 통합을 위한 전처리

In [6]:
df = online_sales.copy()

# customers_data 통합
# [거래날짜] column의 dtype을 datetime으로 변경
df['거래날짜'] = pd.to_datetime(df['거래날짜'])
df = df.merge(customer_data, on='고객ID')
display(df.shape)
display(df.head())


(52924, 12)

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used,남,Chicago,12
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used,남,Chicago,12
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used,남,Chicago,12


In [7]:
# discount_coupon 통합
df['월'] = df['거래날짜'].dt.month
month_to_number = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
    "May": 5, "Jun": 6, "Jul": 7, "Aug": 8,
    "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}

discount_coupon['월'] = discount_coupon['월'].map(month_to_number)
df = df.merge(discount_coupon, on=['월', '제품카테고리'], how='left')

display(df.shape)
display(df.head())
# 추후에 [쿠폰상태]가 Not Used인 행의 [쿠폰코드, 할인율]을 처리해줘야 함

(52924, 15)

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,월,쿠폰코드,할인율
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,1,ELEC10,10.0
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,1,ELEC10,10.0
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used,남,Chicago,12,1,OFF10,10.0
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used,남,Chicago,12,1,SALE10,10.0
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used,남,Chicago,12,1,AIO10,10.0


In [8]:
# tax_amount 통합
df = df.merge(tax_amount, on=['제품카테고리'], how='left')

display(df.shape)
display(df.tail())

(52924, 16)

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,월,쿠폰코드,할인율,GST
52919,USER_0504,Transaction_25056,2019-12-31,Product_0976,Nest-USA,1,121.30,6.50,Clicked,여,New York,45,12,ELEC30,30.0,0.10
52920,USER_0504,Transaction_25057,2019-12-31,Product_0413,Apparel,1,48.92,6.50,Used,여,New York,45,12,SALE30,30.0,0.18
52921,USER_0504,Transaction_25058,2019-12-31,Product_0989,Nest-USA,1,151.88,6.50,Used,여,New York,45,12,ELEC30,30.0,0.10
52922,USER_0562,Transaction_25059,2019-12-31,Product_0985,Nest-USA,5,80.52,6.50,Clicked,여,California,7,12,ELEC30,30.0,0.10
52923,USER_0562,Transaction_25060,2019-12-31,Product_0984,Nest-USA,4,80.52,19.99,Clicked,여,California,7,12,ELEC30,30.0,0.10


In [9]:
# [쿠폰코드, 할인율] null 확인
df.isnull().sum()

고객ID        0
거래ID        0
거래날짜        0
제품ID        0
제품카테고리      0
수량          0
평균금액        0
배송료         0
쿠폰상태        0
성별          0
고객지역        0
가입기간        0
월           0
쿠폰코드      400
할인율       400
GST         0
dtype: int64

In [10]:
# 쿠폰 상태가 used인 상태에만 할인율 적용
df.loc[df['쿠폰상태']!='Used', '할인율'] = 0

In [11]:
# 쿠폰 상태가 used인 상태에만 쿠폰코드 남기기
df.loc[df['쿠폰상태']!='Used', '쿠폰코드'] = None

In [12]:
df['쿠폰코드'].isnull().sum()

35146

[고객소비액, 매출]이란 파생변수를 만든다.  
+ $고객소비액 = 수량 \times 평균금액 \times (1-\frac{할인율}{100}) \times (1+GST) + 배송료$
+ $매출 = 수량 \times 평균금액$

In [ ]:
1 * 119 * (1-10)

In [15]:
# 고객소비액
df['고객소비액'] = df['수량']*df['평균금액']*(1-(df['할인율']/100))*(1+df['GST'])+df['배송료']
df['고객소비액'] = df['고객소비액'].round(2)
# 매출
df['매출'] = (df['수량'] * df['평균금액']).round(2)

df.sample(10)

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,월,쿠폰코드,할인율,GST,고객소비액,매출
40875,USER_1467,Transaction_13451,2019-07-29,Product_0442,Apparel,13,5.10,6.00,Used,여,New Jersey,36,7,SALE10,10.0,0.18,76.41,66.30
9597,USER_1076,Transaction_21706,2019-11-24,Product_0002,Apparel,1,15.99,6.50,Clicked,여,Chicago,26,11,None,0.0,0.18,25.37,15.99
22789,USER_0356,Transaction_5198,2019-03-23,Product_0961,Notebooks & Journals,15,11.99,67.28,Used,남,Chicago,21,3,NJ30,30.0,0.05,199.47,179.85
24295,USER_1397,Transaction_12670,2019-07-18,Product_0869,Housewares,1,1.40,12.99,Clicked,남,Chicago,6,7,None,0.0,0.12,14.56,1.40
878,USER_0736,Transaction_16710,2019-09-12,Product_0981,Nest-USA,1,149.00,6.00,Clicked,여,Chicago,40,9,None,0.0,0.10,169.90,149.00
16345,USER_0443,Transaction_3470,2019-02-23,Product_0186,Headgear,1,24.99,16.65,Clicked,여,Chicago,41,2,None,0.0,0.05,42.89,24.99
30154,USER_1407,Transaction_21469,2019-11-21,Product_0481,Apparel,2,41.59,20.85,Used,여,Chicago,16,11,SALE20,20.0,0.18,99.37,83.18
36496,USER_1334,Transaction_10727,2019-06-18,Product_0989,Nest-USA,2,149.00,6.00,Clicked,여,New York,42,6,None,0.0,0.10,333.80,298.00
240,USER_1358,Transaction_1606,2019-01-24,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,1,ELEC10,10.0,0.10,158.67,153.71
18874,USER_1060,Transaction_4221,2019-03-07,Product_0952,Office,25,8.79,64.01,Not Used,여,New York,40,3,None,0.0,0.10,305.74,219.75


+ 데이터 분석을 위한 df 데이터셋으로 통합함.


## 3. 데이터 분석 (시각화)
### 3-1. 월별 매출/거래량 추이 분석

### 3-2. 월별 마케팅 비용 추이 분석

### 3-3. 마케팅비용과 매출과의 선형관계 분석

### 3-4. 가입기간과 매출과의 관계

### 3-5. 가입연도별 신규고객동향 파악

### 3-6. 지역별 매출/거래량 분석

### 3-7. 제품카테고리별 매출/거래량 분석

## 4. 코호트 분석

### 4-1. 월별 코호트 분석

### 4-2 잔존 빈도 구하기

### 4-3. 잔존율 분석 

### 4-4. 월별 매출액 리텐션 분석

## 5. RFM & k-means를 활용한 고객 세그먼트 정의
+ 데이터의 RFM값과 K-Means 알고리즘을 활용하여 고객을 세분화한다.

### 5-1. RFM 계산

### 5-2. RFM 3D 그래프 분석

### 5-3. 이상치를 제거한 고객의 RFM 3차원 그래프

### 5-4. RFM 값으로 K-Means 클러스터링 수행
+ _
+ _ 
+ _

### 5-5. QQ-Plot을 통한 정규성 검정

### 5-6. K-Means 하이퍼파라미터 K 서칭 & 클러스터링 (엘보우, 실루엣)

### 5-7 클러스터링 된 데이터의 RFM 3차원 그래프

## 6. 클러스터링 된 데이터 분석
### 6-1. RFM 분석

### 6-2. 월 거래량&매출 분석

### 6-3. 고객군별 제품 카테고리 분석

### 6-4. 월별 제품 카테고리 분석

### 6-5. ARPU (지불유저 1명당 한달에 결제하는 평균 금액)

### 6-6.고객군별 지역 분석석

### 6-7. 고객군별 성별 분포

### 6-8. Q. 마케팅 비용이 고객군별 매출/거래량에 어떤 영향을 끼칠까?

### 6-9. 쿠폰 분석